In [1]:
from process_and_ml.data_wrangler import CleanUp, load_azdias, load_customers, load_test_file
from process_and_ml.where_is import WhereIs

In [2]:

where_is = WhereIs(False)

In [3]:
all_paths = where_is.get_paths_list

In [4]:
#load azdias(population info)
azdias_df = load_azdias(all_paths[0])


/home/jair/.cache/pypoetry/virtualenvs/arvatocapstoneproject-L5ipTb8G-py3.7/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3347: DtypeWarning: Columns (18,19) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


In [5]:
customers_df = load_customers(all_paths[1])

In [6]:
clean_up = CleanUp(all_paths)



In [7]:
argument_dict = {'azdias':azdias_df,'customers_df':customers_df}

In [8]:
# Mount dict for pipeline, you can check on pipeline documentation
dfs_list = clean_up.pipeline_clean_up(argument_dict,unknowns_df=all_paths[4], info_df=all_paths[8],attr_df=all_paths[7])

No file exist with columns, ll start cleanup pipeline
/home/jair/Documents/arvato_project/process_and_ml/data_wrangler.py:118: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[key] = df[key].replace(value, np.nan)
/home/jair/.cache/pypoetry/virtualenvs/arvatocapstoneproject-L5ipTb8G-py3.7/lib/python3.7/site-packages/pandas/core/frame.py:4312: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/home/jair/Documents/arvato_project/process_and_ml/data_wrangler.py:118: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

In [9]:
#clean_azdias, clean_customer, clean_test
a_clean, c_clean = dfs_list[0], dfs_list[1]

KeyError: 0

In [ ]:
from models.pipeline import preprocessing_baseline, show_metrics_baseline
from catboost import CatBoostClassifier, Pool


In [ ]:
df_to_model = pd.concat(
    [customers_clean_sample.assign(is_customer=1),
     azdias_clean_sample.assign(is_customer=0)]
)

cat_features = df_to_model.select_dtypes(include=['category', 'object']).columns

features, labels = preprocessing_baseline(df_to_model,
                                          cat_features=cat_features,
                                          target='is_customer')

X_train, X_test, X_valid = features
y_train, y_test, y_valid = labels

class_weights = (1, sum(y_train == 0) / sum(y_train == 1))

catboost = CatBoostClassifier(num_trees=40,
                              max_depth=3,
                              min_child_samples=30,
                              one_hot_max_size=5,
                              class_weights=class_weights,
                              cat_features=cat_features,
                              random_state=RANDOM_STATE,
                              eval_metric='AUC')

catboost.fit(X_train, y_train, eval_set=(X_valid, y_valid), verbose=False)


In [ ]:
show_metrics_baseline(catboost, features=features, labels=labels)

